# Motivación

# Extracción de Tweets

### El objetivo de esta notebook es revisar los pasos para realizar el minado de tweets. Para ello se tocan los siguientes puntos: 
### - Qué es la API de Twitter y cómo utilizarla
### - Herramientas u operadores disponibles para definir queries 
### - Cómo obtener un dataset inicial de tweets 
### - Coómo refinar iterativamente los queries

# Requerimientos



El minado de tweets con Python requiere de descargar previamente algunos programas así como obtener credenciales de la API de Twitter. Cabe señalar que para un minado masivo de tweets, es posible utilizar una credencial de API académica (a la que el Banco tiene acceso), pero para hacer algunas pruebas, cualquier persona puede acceder a una credencial para realizar este minado.

### Credenciales para utilizar la API de Twitter

Twitter provee una API (interfaz de programación de aplicaciones) para acceder a sus datos. Las API son mecanismos que permiten a dos componentes de software comunicarse entre sí mediante un conjunto de definiciones y protocolos. Por ejemplo, el sistema de software del instituto de meteorología contiene datos meteorológicos diarios. La aplicación meteorológica de su teléfono “habla” con este sistema a través de las API y le muestra las actualizaciones meteorológicas diarias en su teléfono. [Aqui una explicación de AWS](https://aws.amazon.com/es/what-is/api/).

Para obtener tweets mediante la [API Oficial de Twitter](https://developer.twitter.com/en/docs/twitter-api) es necesario registrarse en el [portal para desarrolladores de Twitter](https://developer.twitter.com/en/docs/developer-portal/overview) para así dar de alta un proyecto y obtener las [credenciales necesarias](https://developer.twitter.com/en/docs/twitter-api/getting-started/getting-access-to-the-twitter-api).

Es importante señalar que **el tipo de credenciales y el método de autenticación empleado afectan la [cantidad de tweets que podremos obtener en cierto tiempo -días hacia atrás; cantidad máxima de tweets por mes](https://developer.twitter.com/en/docs/twitter-api/rate-limits#v2-limits), así como los [operadores/filtros que podemos usar] y su extensión** (https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list)  

Por ejemplo, los límites mensuales de obtención de tweets, dependiendo de la credencial empleada, son los siguientes:
- *Essential*: 500k tweets en un mes y 512 caracteres por query.
- *Elevated*: 2M tweets en un mes y 512 caracteres por query.
- *Academic*: 10M tweets en un mes y 1024 caracteres por query.

Cualquier otra diferencia entre nivel de credenciales, se puede [ver aquí](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level)

Notas:
- Debido a que la webapp del Laboratorio de Migración solo necesita permisos de lectura para contenido público, nos interesa obtener la credencial de tipo **Bearer Token**, ya que es el método de autenticación que más tweets nos permite obtener en menos tiempo.
- Se usará la versión 2 de la [API de Twitter](https://developer.twitter.com/en/docs/twitter-api).
- El ejercicio aquí propuesto puede realizarse con las credenciales de nivel [*Elevated*](https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-level), pero en caso de estar trabajando en un proyecto con fines académicos o de estudio, se puede [aplicar de manera gratuita a credenciales Académicas](https://developer.twitter.com/en/products/twitter-api/academic-research) y así trabajar con límites más amplios y operadores avanzados.

### Dependencias de Python
Las dependencias que se utilizan para la extracción de tweets son `pandas`, `numpy` y  `twarc`. Las primeras dos se utilizan para manipular y trabajar con los datos, mientras que [`twarc` es el paquete](https://developer.twitter.com/en/docs/twitter-api/rate-limits)  que nos facilitará el proceso de minado, ya que se encarga de obtener automáticamente todos los atributos de los tweets así como de manejar los tiempos de espera cuando se llega a los límites de minado [de la API](https://developer.twitter.com/en/docs/twitter-api/rate-limits).

In [3]:
# Indispensables
!pip install twarc==2.9.2
!pip install pandas==1.4.1
!pip install numpy==1.22.3

# Para mejorar la interacción gráfica
!pip install tqdm==4.62.2
!pip install pendulum==2.1.2

     |████████████████████████████████| 55 kB 1.5 MB/s eta 0:00:011
     |████████████████████████████████| 106 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 16.2 MB/s eta 0:00:01
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34528 sha256=18d7be81f0d7569b76771d8ccd82b7fe5d9c25954885b1ddd735b4b06ed765a8
  Stored in directory: /Users/aleistermontfort/Library/Caches/pip/wheels/90/cd/58/816b023d3eb0e2c9c6e0cd834a10a53e932e29d7c719298216
Successfully built configobj
     |████████████████████████████████| 11.5 MB 2.2 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
skforecast 0.4.3 requires numpy<=1.22,>=1.20, but you have numpy 1.22.2 which is 

## Imports & Credenciales

In [4]:
import os
import pandas as pd
from tqdm import tqdm

from twarc.client2 import Twarc2
from twarc.expansions import TWEET_FIELDS
from twarc.expansions import ensure_flattened

Ingresa tu propio Bearer Token o comentalo, y descomenta el resto de atributos para con tus propias API Keys y Access Tokens.

In [5]:
CREDENTIALS_TWITTER = {
    'bearer_token': "Enter your own Bearer token",

    # 'api_key': "Enter your own API Key",
    # 'api_secret_key': "Enter your own API Secret Key",
    # 'access_token': "Enter your own access_token",
    # 'access_token_secret': "Enter your own access_token_secret"
}

CREDENTIALS_TWITTER = {
    'bearer_token': "AAAAAAAAAAAAA0AAAAAAAAFBIPwEAAAAAxt1ACpkLKEy%2FwL4KvhyxKCLAIbA%3D1ORuBxC0mD6fY47zNqdNAXICGQboR5EQu0hxie6P3EAeEFo6ZW"
}

## Minado de Tweets

La API de Twitter permite obtener diferentes piezas de información a partir de los usuarios y los tweets que publican, según el tipo de operadores y queries utilizados. Para fines de esta notebook, nos centraremos en obtener tweets públicos en un intervalo de tiempo definido, a partir del contenido de palabras claves.

Las palabras claves del tema que se desea minar se utilizan para construir queries que hacen la búsqueda más certera.


### Endpoints & Limites

Un punto final o Endopint es un dispositivo informático remoto que se comunica a través de una red a la que está conectado. Normalmente se refiere a los dispositivos que utilizamos a diario como ordenadores de escritorio, portátiles, teléfonos inteligentes, tablets o dispositivos de Internet de las cosas (IoT).

Para [buscar tweets](https://developer.twitter.com/en/docs/twitter-api/tweets/search/introduction) con la API de Twitter existen dos endpoints: [Recent Search](https://developer.twitter.com/en/docs/twitter-api/tweets/search/quick-start/recent-search) y  [Full Archive](https://developer.twitter.com/en/docs/twitter-api/tweets/search/quick-start/full-archive-search).
- **Recent Search**: Nos permite realizar 450 requests (pedidos) a la API en una ventana de 15 minutos con un máximo de 100 tweets por request. Sin embargo, solo se pueden obtener tweets publicados en los últimos 7 días.
- **Full Archive**: Podremos obtener tweets publicados desde el inicio de la red social, pero solo se puede acceder a 300 requests en una ventana de 15 minutos, con hasta 500 tweets por cada request. Hay que aplicar a las [credenciales academicas](https://developer.twitter.com/en/products/twitter-api/academic-research) para poder utilizar este endpoint.

PREGUNTA: un request cuántos tweets te permite jalar? Un request == 1 query?

**Nota**: No hay que preocuparse del código de error que aparece una vez que el minado alcanza estos límites, pues el paquete twarc se encarga de pausar la obtención de tweets una vez que se llega al limite de 450 (o 300) requests en la ventana de 15 minutos; una vez que pasa un tiempo necesario, twarc reanuda el proceso. Si se llega al límite de tweets en un mes, la función para.



### Operadores

En esta sección vamos a ver de manera general qué son  y cómo funcionan los operadores, pero la lista y descripción completa de los mismos se puede [encontrar aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query).

Existen dos tipos de operadores: 
- **Standalone**: Se pueden usar solos o junto con otros. Por ejemplo, buscar tweets con un hashtag en especifico: `#migrantes`.
- **Conjunction-required**: Es necesario que estén junto a mínimo un operador *standalone*. Por ejemplo, buscar tweets con un hastag en especifico pero que incluyan imagenes y sean retweets: `#migrantes has:media is:retweet`

Además, como se mencionó en la sección de "Credenciales para la API de Twitter", existen operadores *core*, que son accesibles con cualquier nivel de acceso, así como los operadores *advanced*, que solo se pueden utilizar con un acceso académico. 

#### Operadores Lógicos

- **AND**: Obtiene tweets que cumplan con los dos operadores, se logra dejando un espacio en blanco entre ellos. 
    - Ejemplo, obtener tweets que contienen la palabra *politicos* y el hashtag *#corruptos*: `politicos #corruptos`.
- **OR**: Obtiene tweets que cumplan con alguno de los dos operadores. Hay que añadir el string " OR " entre los operadores. 
    - Ejemplo, tweets que contengan la palabra *migrantes* o *inmigrantes*: `migrantes OR inmigrantes`.
- **NOT**: Obtiene tweets que no contengan el operador o la keyword negada. Se logra añadiendo un guión medio "-" antes del operador. 
    - Ejemplo, obtener tweets con la palabra *politicos* pero sin la palabra *corruptos*: `politicos -corruptos`
    - Ejemplo, obtener tweets con la palabra *migrantes pero que no sean retweets*: `migrantes -is:retweets`
- **Grouping**: Sirve para agrupar operadores logicos, y hay que encerrar los operadores entre paréntesis. Un grupo no puede ser negado.
    - Ejemplo, obtener tweets con la palabra *migrantes* y alguna de las palabras *llegan* o *salen*: `migrantes AND (llegan OR salen)`

**Nota**: A menos de que haya paréntesis para especificar el orden de operadores, primero se resuelven aquellos que son *AND* y luego los *OR*. [Más aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#boolean).

#### Más Operadores

- **keyword**: Hace match con tweets que contengan un string en especifico (no se pueden usar wildcards ····· DEFINIR WILDCARDS······). No es sensible a caracteres en mayúsculas o minúsculas; acentos caracteres especiales como ñ. Ejemplo: `migrantes OR inmigrantes`.
- **"exact phrase"**: Parecido al anterior, permite considerar espacios y múltiples tokens. Tiene que estar entre comillas dobles. Ejemplo: `"ola migrante"`.
- **#**: Hace match a tweets que tengan el hashtag incluido. Ejemplo: `#migraresunderecho`.
- **@**: Hace match a tweets que mencionen a los usuarios incluidos. Ejemplo: `@IADB`.
- **place_country**: Obtiene tweets que sean geolocalizables a cierto pais. Hay que pasarle el código [ISO del país](https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2). Ejemplo: `place_country:MX`.
- **lang**: Obtiene tweets que esten escrito en el lenguaje indicado y tiene que estar junto con un standalone operator. Ejemplo: `migrantes lang:es`.
- **is:retweet**: Obtiene solo tweets que sean retweets y tiene que estar junto con un standalone operator. Ejemplo: `migrantes is:retweet`.

La lista completa de operadores esta [aquí](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#list).

#### Ejercicio de Minado

La siguiente función recibe una lista de queries y un intervalo de tiempo para guardar los tweets en un archivo JSONL.

**Nota**: Un archivo de extension .jsonl guarda un objeto json por cada linea separados por solo el salto de linea, no comas u otro separador.

In [ ]:
def get_tweets(twarc_config,
               queries_list,
               output_file,
               since_date, until_date,
               is_academic=False):
    """Function to scrape tweets from the 
    official Twitter API, using the library "Twarc".

    Args:
        twarc_config (dict): Dictionary with the Twitter API credentials.
        queries_list (list[str]): List of queries we want to scrape.
        output_file (str): Path to the file where we store the results.
        since_date (datetime): Initial date to scrape.
        until_date (datetime): Last date to scrape.
        is_academic (bool, optional): If the credentials have Research 
                                      Academic access level.
    """

    # Instiate the Twarc Client
    twarc_client = Twarc2(**twarc_config)

    # Make some tweaks for using the research credentials
    max_size = 100
    tweet_fields = TWEET_FIELDS.copy()
    search_func = twarc_client.search_recent
    if(is_academic):
        search_func = twarc_client.search_all
        max_size = 500

        # Remove the context_annotations attr to
        # scrape 500 tweets per request
        tweet_fields.remove('context_annotations')

    tweet_fields = ','.join(tweet_fields)

    # Keep in memory the wanted attrs
    return_values = []

    with open(output_file, 'a') as pages_file:
        for query in tqdm(queries_list):

            search_results = search_func(query=query,
                                         start_time=since_date,
                                         end_time=until_date,
                                         tweet_fields=tweet_fields,
                                         max_results=max_size)

            # Write all the obtained tweets
            for page in search_results:

                # Write one by one the tweets
                for tweet in ensure_flattened(page):
                    json.dump(tweet, pages_file)
                    pages_file.write('\n')

Se declara la lista de queries, el intervalo de fecha (para fines de la prueba, es 6hrs hacia atras de la hora actual) y si las credenciales que se estan utilizando tienen un nivel de acceso academico. Esto ultimo es util para pedir que cada request tenga 500 tweets.

In [ ]:
lst_queries = ['migrantes', 'inmigrantes', 'emigrantes']

date_start = pendulum

